In [1]:
# imports
%matplotlib inline
import seaborn as sns
from pyrt.data.data_trots import *
from pyrt.optimization.vmat import *
from pyrt.tools import print_structure_info


In [2]:
# Troy Inputs - Prostate
# Find work directory
# cwd = '/Users/troy/Dropbox/CAP Group/TROTS/Prostate_VMAT/'
working_directory = 'outputs_troy/'
cwd = '/media/troy/DataDrive/Dropbox/CAP Group/TROTS/Prostate_VMAT/'

# Load patient information
filename = 'Prostate_VMAT_101.mat'

imrt_model_params = {
    'target_weights':{'PTV':1000., 'default':500.},
    'oar_weights':{'Rectum':10., 'Bladder':10., 'default':1.}
}


input_dict = {
    'cwd': cwd,
    'figure_directory':working_directory,
    'filename': filename,
    'Rx': {'PTV': 79.56, 'PTV Vesicles': 72.2},
    'model_params':imrt_model_params
}



In [3]:
# Troy Inputs - H&N
# Find work directory
# cwd = '/Users/troy/Dropbox/CAP Group/TROTS/Prostate_VMAT/'
working_directory = 'outputs_troy/'
cwd = '/media/troy/DataDrive/Dropbox/CAP Group/TROTS/Prostate_VMAT/'

# Load patient information
filename = 'Prostate_VMAT_101.mat'

imrt_model_params = {
    'target_weights':{'PTV':1000., 'default':500.},
    'oar_weights':{'Rectum':10., 'Bladder':10., 'default':1.}
}


input_dict = {
    'cwd': cwd,
    'figure_directory':working_directory,
    'filename': filename,
    'Rx': {'PTV': 79.56, 'PTV Vesicles': 72.2},
    'model_params':imrt_model_params
}

In [4]:
model = vmat_mip(input_dict)

---------------------
|  Reading in data  |
---------------------
[ 85.  88.  89.  88.  87.  83.  79.  83.  85.  88.  88.  87.  89.  90.  88.
  85.  83.  80.  81.  86.  88.  86.  86.]
[    0.    85.   173.   262.   350.   437.   520.   599.   682.   767.
   855.   943.  1030.  1119.  1209.  1297.  1382.  1465.  1545.  1626.
  1712.  1800.  1886.  1972.]
importing PTV Dij as dense matrix, converting to sparse...
importing PTV Shell 50 mm Dij as dense matrix, converting to sparse...
importing External Ring 20 mm Dij as sparse matrix
importing PTV Shell 5 mm Dij as dense matrix, converting to sparse...
importing PTV Shell 15 mm Dij as dense matrix, converting to sparse...
importing PTV Shell 25 mm Dij as dense matrix, converting to sparse...
importing Rectum Dij as dense matrix, converting to sparse...
importing Anus Dij as dense matrix, converting to sparse...
importing Hip (L) Dij as dense matrix, converting to sparse...
importing Hip (R) Dij as dense matrix, converting to sparse...
imp

In [6]:
cp0 = model.data.control_points[0]
assert(isinstance(cp0,control_point))
for r in range(cp0.num_rows):
    print cp0.field[cp0.min_row+r][cp0.left_leaf_position[r]:cp0.left_leaf_position[r]+cp0.num_beamlets_in_row[r]]

[1 2 3 4 5 6 7]
[ 8  9 10 11 12 13 14 15 16 17]
[18 19 20 21 22 23 24 25 26 27 28 29]
[30 31 32 33 34 35 36 37 38 39 40 41 42]
[43 44 45 46 47 48 49 50 51 52 53 54 55 56 57]
[58 59 60 61 62 63 64 65 66 67 68 69 70 71 72]
[73 74 75 76 77 78 79 80 81 82 83 84 85]


In [40]:
model.data.structures[0].Dij[5,:].todense()
indices = np.array([1,2])
a = np.asarray(model.data.structures[0].Dij[indices].sum(axis=0))

b = (model.data.structures[0].Dij[1,:] + model.data.structures[0].Dij[2,:]).transpose().todense()

print a.flatten()
print b.flatten()
a[0]

[  1.26426439e-05   1.25645802e-05   1.31702936e-05 ...,   8.88721934e-06
   9.88377178e-06   9.92617970e-06]
[[  1.26426439e-05   1.25645802e-05   1.31702936e-05 ...,   8.88721934e-06
    9.88377178e-06   9.92617970e-06]]


array([  1.26426439e-05,   1.25645802e-05,   1.31702936e-05, ...,
         8.88721934e-06,   9.88377178e-06,   9.92617970e-06], dtype=float32)